# DAT210x - Programming with Python for DS

## Module5- Lab7

In [1]:
import random, math
import pandas as pd
import numpy as np
import scipy.io

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

plt.style.use('ggplot') # Look Pretty


# Leave this alone until indicated:
Test_PCA = False

### A Convenience Function

This method is for your visualization convenience only. You aren't expected to know how to put this together yourself, although you should be able to follow the code by now:

In [2]:
def plotDecisionBoundary(model, X, y):
    print("Plotting...")

    fig = plt.figure()
    ax = fig.add_subplot(111)

    padding = 0.1
    resolution = 0.1

    #(2 for benign, 4 for malignant)
    colors = {2:'royalblue', 4:'lightsalmon'} 


    # Calculate the boundaris
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_min -= x_range * padding
    y_min -= y_range * padding
    x_max += x_range * padding
    y_max += y_range * padding

    # Create a 2D Grid Matrix. The values stored in the matrix
    # are the predictions of the class at at said location
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                         np.arange(y_min, y_max, resolution))

    # What class does the classifier say?
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the contour map
    plt.contourf(xx, yy, Z, cmap=plt.cm.seismic)
    plt.axis('tight')

    # Plot your testing points as well...
    for label in np.unique(y):
        indices = np.where(y == label)
        plt.scatter(X[indices, 0], X[indices, 1], c=colors[label], alpha=0.8)

    p = model.get_params()
    plt.title('K = ' + str(p['n_neighbors']))
    plt.show()

### The Assignment

Load in the dataset, identify nans, and set proper headers. Be sure to verify the rows line up by looking at the file in a text editor.

In [3]:
X = pd.read_csv('Datasets/breast-cancer-wisconsin.data', names = ['sample', 'thickness', 'size', 'shape', 'adhesion', 'epithelial', 'nuclei', 'chromatin', 'nucleoli', 'mitoses', 'status'])

X.nuclei = pd.to_numeric(X.nuclei, errors = 'coerce')
X = X.fillna(method = 'backfill', axis = 1)
print(X)

        sample  thickness  size  shape  adhesion  epithelial  nuclei  \
0    1000025.0        5.0   1.0    1.0       1.0         2.0     1.0   
1    1002945.0        5.0   4.0    4.0       5.0         7.0    10.0   
2    1015425.0        3.0   1.0    1.0       1.0         2.0     2.0   
3    1016277.0        6.0   8.0    8.0       1.0         3.0     4.0   
4    1017023.0        4.0   1.0    1.0       3.0         2.0     1.0   
5    1017122.0        8.0  10.0   10.0       8.0         7.0    10.0   
6    1018099.0        1.0   1.0    1.0       1.0         2.0    10.0   
7    1018561.0        2.0   1.0    2.0       1.0         2.0     1.0   
8    1033078.0        2.0   1.0    1.0       1.0         2.0     1.0   
9    1033078.0        4.0   2.0    1.0       1.0         2.0     1.0   
10   1035283.0        1.0   1.0    1.0       1.0         1.0     1.0   
11   1036172.0        2.0   1.0    1.0       1.0         2.0     1.0   
12   1041801.0        5.0   3.0    3.0       3.0         2.0    

Copy out the status column into a slice, then drop it from the main dataframe. Always verify you properly executed the drop by double checking (printing out the resulting operating)! Many people forget to set the right axis here.

If you goofed up on loading the dataset and notice you have a `sample` column, this would be a good place to drop that too if you haven't already.

In [4]:
y = X.status
X.drop(labels = ['sample', 'status'], axis = 1, inplace = True)

With the labels safely extracted from the dataset, replace any nan values with the mean feature / column value:

In [5]:
from sklearn import preprocessing
T = preprocessing.normalize(X)

Do train_test_split. Use the same variable names as on the EdX platform in the reading material, but set the random_state=7 for reproducibility, and keep the test_size at 0.5 (50%).

In [6]:
from sklearn.cross_validation import train_test_split
data_train, data_test, label_train, label_test = train_test_split(X, y, test_size = 0.5, random_state = 7)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Experiment with the basic SKLearn preprocessing scalers. We know that the features consist of different units mixed in together, so it might be reasonable to assume feature scaling is necessary. Print out a description of the dataset, post transformation. Recall: when you do pre-processing, which portion of the dataset is your model trained upon? Also which portion(s) of your dataset actually get transformed?

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 4, weights = 'distance')
knn.fit(data_train, label_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='distance')

### Dimensionality Reduction

PCA and Isomap are your new best friends

In [9]:
model = None
if Test_PCA:
    print("Computing 2DPrincipal Components")
    #
    # TODO: Implement PCA here. save your model into the variable 'model1'.
    # You should reduce down to two dimensions.
    #
    # .. your code here ..
    model = PCA(n_components = 2)   

else:
    print("Computing 2D Isomap Manifold")
    #
    # TODO: Implement Isomap here. save your model into the variable 'model1'
    # Experiment with K values from 5-10.
    # You should reduce down to two dimensions.
    #
    # .. your code here ..
    model = manifold.Isomap(n_neighbors = 4, n_components = 2)

Computing 2D Isomap Manifold


NameError: name 'manifold' is not defined

Train your model against data_train, then transform both `data_train` and `data_test` using your model. You can save the results right back into the variables themselves.

In [10]:
print(knn.score(data_test, label_test))

0.9628571428571429


Implement and train `KNeighborsClassifier` on your projected 2D training data here. You can name your variable `knmodel`. You can use any `K` value from 1 - 15, so play around with it and see what results you can come up. Your goal is to find a good balance where you aren't too specific (low-K), nor are you too general (high-K). You should also experiment with how changing the weights parameter affects the results.

In [ ]:
# .. your code here ..

Be sure to always keep the domain of the problem in mind! It's WAY more important to errantly classify a benign tumor as malignant, and have it removed, than to incorrectly leave a malignant tumor, believing it to be benign, and then having the patient progress in cancer. Since the UDF weights don't give you any class information, the only way to introduce this data into SKLearn's KNN Classifier is by "baking" it into your data. For example, randomly reducing the ratio of benign samples compared to malignant samples from the training set.

Calculate and display the accuracy of the testing set:

In [ ]:
# .. your code changes above ..

In [ ]:
plotDecisionBoundary(knmodel, X_test, y_test)